# Does a certain area (location) have higher/lower adoption rates?

In [1]:
import pandas as pd
import numpy as np
import requests
import petpy
import pprint

from matplotlib import pyplot as plt
from config import api_key as api_key
from config import api_secret as api_secret

## Retrive data from APIs

**API/Resources:**
- [Adopt a Pet API](https://www.adoptapet.com/public/apis/pet_list.html)
- [Pet Finder API](https://www.petfinder.com/developers/)


In [2]:
pf = petpy.Petfinder(key=api_key, secret=api_secret)

In [3]:
#2018 - 2023 adoptable dogs
adoptable_dogs = pf.animals(animal_type = 'dog', status='adoptable', location='NC', pages=None, sort='recent', after_date='2017-12-31', before_date='2022-01-01', results_per_page=100, return_df=True)
print('Adoptable dogs: '+ str(len(adoptable_dogs['id'].unique())))

#2018 - 2023 adoptable cats
adoptable_cats = pf.animals(animal_type = 'cat', status='adoptable', location='NC', pages=None, sort='recent', after_date='2017-12-31', before_date='2022-01-01', results_per_page=100, return_df=True)
print('Adoptable cats: '+ str(len(adoptable_cats['id'].unique())))

Adoptable dogs: 282
Adoptable cats: 335


In [5]:
#Clean Data
to_drop = ['url', 'species', 'age', 'gender', 'size', 'coat', 'tags', 
           'animal_id', 'animal_type', 'primary_photo_cropped', 'photos', 
           'videos', 'primary_photo_cropped.full', 'primary_photo_cropped.large', 
           'primary_photo_cropped.small', 'primary_photo_cropped.medium','name', 
           'description', 'organization_animal_id', 'contact.email', 'contact.phone', 
           'breeds.primary', 'breeds.secondary', 'breeds.mixed', 'breeds.unknown',
           'colors.primary', 'colors.secondary', 'colors.tertiary',
           'attributes.spayed_neutered', 'attributes.declawed', 'attributes.special_needs',
           'attributes.shots_current', 'attributes.house_trained',
           'environment.children', 'environment.dogs', 'environment.cats']

adoptable_dogs.drop(to_drop, inplace = True, axis=1)
adoptable_cats.drop(to_drop, inplace = True, axis=1)


In [6]:
#Drop last column - duplicate organization ID
adoptable_dogs_clean = adoptable_dogs.iloc[: , :-1]
adoptable_cats_clean = adoptable_cats.iloc[: , :-1]

#Rename Columns
new_columns = {
    'id':'ID',
    'organization_id':'Organization ID',
    'type':'Type',
    'status':'Status',
    'status_changed_at':'Status Changed',
    'published_at':'Published',
    'distance':'Distance',
    'contact.address.address1':'Address 1',
    'contact.address.address2':'Address 2',
    'contact.address.city':'City',
    'contact.address.state':'State',
    'contact.address.postcode':'Zip Code',
    'contact.address.country':'Country'
}

adoptable_dogs_clean.rename(columns=new_columns, inplace=True)
adoptable_cats_clean.rename(columns=new_columns, inplace=True)

#Print New DFs
print(adoptable_dogs_clean.head())
adoptable_cats_clean.head()

         ID Organization ID Type     Status            Status Changed  \
0  54050425           SC170  Dog  adoptable  2021-12-30T02:22:21+0000   
1  54045877            NC11  Dog  adoptable  2022-12-01T17:16:46+0000   
2  54031867           NC419  Dog  adoptable  2022-03-07T17:20:52+0000   
3  54022085          NC1172  Dog  adoptable  2021-12-27T20:21:46+0000   
4  54005089           NC698  Dog  adoptable  2021-12-24T20:42:08+0000   

                  Published  Distance          Address 1  \
0  2021-12-30T02:22:21+0000   83.3665  1705 Adoption Way   
1  2021-12-29T19:47:02+0000   95.3447        PO Box 1884   
2  2021-12-28T17:13:33+0000   81.6870               None   
3  2021-12-27T20:21:46+0000   49.6520               None   
4  2021-12-24T20:42:02+0000   90.9259        PO Box 2746   

                              Address 2         City State Zip Code Country  
0                                  None   Darlington    SC    29532      US  
1  Adoption Center at 417 Kirksey Drive    M

,ID,Organization ID,Type,Status,Status Changed,Published,Distance,Address 1,Address 2,City,State,Zip Code,Country
0,54071180,VA609,Cat,adoptable,2022-01-01T02:29:03+0000,2022-01-01T02:29:03+0000,79.8959,None,None,Stuart,VA,24171,US
1,53995527,NC1111,Cat,adoptable,2021-12-23T18:36:30+0000,2021-12-23T18:36:28+0000,85.1958,None,None,Raleigh,NC,27616,US
2,53995421,NC1111,Cat,adoptable,2021-12-23T18:22:53+0000,2021-12-23T18:22:50+0000,85.1958,None,None,Raleigh,NC,27616,US
3,53951752,NC929,Cat,adoptable,2021-12-19T14:59:27+0000,2021-12-19T14:59:27+0000,51.7933,None,None,Burlington,NC,27215,US
4,53890155,NC1070,Cat,adoptable,2021-12-13T20:39:24+0000,2021-12-13T20:39:24+0000,57.8536,None,None,Pineville,NC,28134,US


In [10]:
#Adoptable Dogs in NC from 2018 - 2022
adoptable_dogs_nc = adoptable_dogs_clean.loc[adoptable_dogs_clean['State'] == 'NC']
adoptable_dogs_nc['City'].value_counts()

Raleigh          32
Charlotte        15
Holly Springs    12
Monroe           10
Thomasville      10
Lenoir           10
Mooresville       8
Albemarle         8
Staley            8
Winston Salem     7
Sanford           6
Waxhaw            5
Shelby            5
Snow Camp         5
Morganton         4
Matthews          4
Pineville         3
Siler City        3
Conover           3
Carthage          2
Greensboro        2
Youngsville       2
Selma             2
Chapel Hill       2
Elizabethtown     2
Cary              2
Mount Airy        2
Claremont         2
Vass              2
Smithfield        2
Dunn              2
Rougemont         1
Franklinton       1
Locust            1
Huntersville      1
Fayetteville      1
Kannapolis        1
Sophia            1
Walnut Cove       1
Mint Hill         1
Durham            1
Bahama            1
Whitsett          1
Julian            1
Concord           1
Garner            1
Pittsboro         1
Maiden            1
Name: City, dtype: int64

In [14]:
adoptable_dogs_nc['Organization ID'].value_counts()

NC183     21
NC457     12
NC1022    10
NC698     10
NC626     10
          ..
NC1152     1
NC150      1
WV178      1
NC975      1
VA761      1
Name: Organization ID, Length: 68, dtype: int64

In [ ]:
#Bar Graph of 5+ adoptable dogs in NC Cities

In [11]:
#Adoptable Cats in NC from 2018 - 2022
adoptable_cats_nc = adoptable_cats_clean.loc[adoptable_cats_clean['State'] == 'NC']
adoptable_cats_nc['City'].value_counts()

Pittsboro        89
Burlington       27
Greensboro       25
Raleigh          18
Vass             13
Carthage         12
Charlotte         9
Salisbury         8
Apex              7
Morganton         7
Youngsville       6
Winston-Salem     6
Cary              5
Liberty           5
Albemarle         5
Fayetteville      4
Toast             3
Statesville       3
Lincolnton        3
Lexington         3
Smithfield        3
Garner            3
Pineville         2
Concord           2
Rockingham        2
Zebulon           2
Selma             2
Thomasville       1
Trinity           1
Mint Hill         1
Indian Trail      1
Oakboro           1
Dunn              1
Conover           1
Name: City, dtype: int64

In [15]:
adoptable_cats_nc['Organization ID'].value_counts()

NC552     82
NC929     26
NC1019    13
NC107     10
NC1111     9
NC936      8
NC519      8
NC1057     7
NC1016     7
NC21       7
NC915      7
NC72       7
NC626      6
NC776      6
NC518      5
NC399      5
NC1109     4
NC591      4
NC1025     3
NC1020     3
NC1142     3
NC1122     3
NC548      3
NC122      3
NC422      3
NC1156     3
NC631      3
NC1071     2
NC1149     2
NC282      2
NC1070     2
NC87       2
NC979      2
NC661      2
NC82       2
NC697      1
NC773      1
NC851      1
NC840      1
NC103      1
NC210      1
NC1106     1
NC986      1
NC88       1
NC419      1
NC854      1
NC1078     1
NC65       1
NC1174     1
NC880      1
NC1005     1
NC757      1
Name: Organization ID, dtype: int64

In [ ]:
#Bar Graph of 5+ adoptable dogs in NC Cities

In [13]:
#Adoptable Cats & Dogs in NC from 2018 - 2022
adoptable_cats_dogs = pd.merge(adoptable_cats_nc, adoptable_dogs_nc, on='Status')
adoptable_cats_dogs.head()

'''adoptable_cats_nc = adoptable_cats_clean.loc[adoptable_cats_clean['State'] == 'NC']
adoptable_cats_nc['City'].value_counts()'''

"adoptable_cats_nc = adoptable_cats_clean.loc[adoptable_cats_clean['State'] == 'NC']\nadoptable_cats_nc['City'].value_counts()"

In [ ]:


#Data Frame of only NC organizations
nc_organizations_df = pf.organizations(state = 'NC', return_df=True)


#CLean DataFrame
to_drop = ['email', 'phone', 'url', 'photos', 'distance', 'organization_id','address.address2','address.country']
nc_organizations_df.drop(to_drop, inplace = True, axis=1)


#Rename Columns
new_columns = {
    'name': 'Name',
    'website': 'Website',
    'mission_statement':'Misson Statement',
    'address.address1': 'Address',
    'address.city':'City',
    'address.state':'State',
    'address.postcode':'Zip',
    'hours.friday':'Friday',
    'hours.saturday':'Saturday',
    'hours.sunday':'Sunday',
    'adoption.policy':'Adoption Policy',
    'adoption.url':'Adoption URL',
    'social_media.facebook':'Facebook',
    'social_media.twitter':'Twitter',
    'social_media.youtube':'YouTube',
    'social_media.instagram':'Instagram',
    'social_media.pinterest':'Pinterest'
}
nc_organizations_df.rename(columns=new_columns, inplace=True)
nc_organizations_df = nc_organizations_df.set_index('id')

#Data Frame of Organizations in NC
nc_organizations_df.head()

In [ ]:
#Daily Schedule of organizations
'''towns_df = pd.DataFrame(university_towns,
...                         columns=['State', 'RegionName'])'''
schedule_df = pd.DataFrame(nc_organizations_df, columns = ['Friday', 'Saturday', 'Sunday'])
schedule_df